In [ ]:
!pip install -q -U huggingface huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q scipy
# install trl for the SFT library
!pip install -q trl


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#model_id = 'bigcode/starcoderbase-7b'
#model_id = 'codellama/CodeLlama-7b-hf'
model_id = 'bigcode/starcoderbase-1b'
new_model = "starcoder-1b-taipy30" # You must create this model so that at the end the fine-tuned model will be pushed to
dataset = "thanhnew2001/taipydata25"

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
!pip install chardet

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
from datasets import load_dataset

data = load_dataset(dataset)
data["train"] = data["train"].map(lambda sample: {"text": "<s>[INST]" + sample["prompt"] + "[/INST]\n" + sample["Markdown code"] + "</s>"})
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
# install trl for the SFT library
!pip install -q trl

In [ ]:
import transformers
from trl import SFTTrainer

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    dataset_text_field="text",
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_train

Step,Training Loss
1,4.182800
2,4.423500
3,4.660300
4,4.121600
5,4.293000
6,4.271600
7,3.963600
8,4.050200
9,4.109900
10,4.194200


TrainOutput(global_step=10, training_loss=4.227058482170105, metrics={'train_runtime': 18.5805, 'train_samples_per_second': 4.306, 'train_steps_per_second': 0.538, 'total_flos': 20820288307200.0, 'train_loss': 4.227058482170105, 'epoch': 0.034482758620689655})

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)


In [ ]:
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

NameError: name 'model' is not defined

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, PeftModel

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

In [ ]:
# Test the new model
inputs = tokenizer.encode("<s>[INST]create a slider 10 - 100</s>[/INST]", return_tensors="pt").to("cuda")
outputs = model.generate(inputs,  max_length=64)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s>[INST]create a slider 10 - 100</s>[/INST]
<s>[INST]create a slider 100 - 1000</s>[/INST]
<s>[INST]create a slider 1000 - 1


In [ ]:
# Test the new model
inputs = tokenizer.encode("<s>[INST]create a button</s>[/INST]", return_tensors="pt").to("cuda")
outputs = model.generate(inputs,  max_length=64)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s>[INST]create a button</s>[/INST]
<s>[INST]create a button</s>[/INST]
<s>[INST]create a button</s>[/INST]
<s>[INST]create a button</s>[/INST]
<s>[INST


In [ ]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/thanhnew2001/starcoder-1b-taipy30/commit/639d0b0c08fb3094776cae33b4641deee333cc0b', commit_message='Upload tokenizer', commit_description='', oid='639d0b0c08fb3094776cae33b4641deee333cc0b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/thanhnew2001/starcoder-1b-taipy30', endpoint='https://huggingface.co', repo_type='model', repo_id='thanhnew2001/starcoder-1b-taipy30'), pr_revision=None, pr_num=None)